In [78]:
import pandas as pd

df = pd.read_csv('dutch_housing_data.csv')

In [79]:
df.head()

,Address,City,Price,Lot size (m2),Living space size (m2),Build year,Build type,House type,Roof,Rooms,Toilet,Floors,Energy label,Position,Garden,Estimated neighbourhood price per m2
0,Bovenweg 223,Sint Pancras,€ 525.000,251 m²,135 m²,1972,Bestaande bouw,"Eengezinswoning, 2-onder-1-kapwoning",Zadeldak bedekt met pannen,5 kamers (4 slaapkamers),1 badkamer en 1 apart toilet,3 woonlagen,D,NaN,"Achtertuin, voortuin en zijtuin",6.035
1,Julianastraat 19,Waddinxveen,€ 425.000,181 m²,109 m²,1935,Bestaande bouw,"Herenhuis, geschakelde 2-onder-1-kapwoning",Schilddak bedekt met bitumineuze dakbedekking ...,4 kamers (3 slaapkamers),1 badkamer en 1 apart toilet,3 woonlagen,E,Aan rustige weg en in woonwijk,Achtertuin en voortuin,1.435
2,Kuipersgilde 3,Houten,€ 575.000,198 m²,138 m²,1986,Bestaande bouw,"Eengezinswoning, 2-onder-1-kapwoning",Zadeldak bedekt met pannen,5 kamers (4 slaapkamers),1 badkamer en 1 apart toilet,3 woonlagen,B,Aan rustige weg en in woonwijk,Achtertuin,1.855
3,De Kamp 1 A,Pannerden,€ 259.500,231 m²,92 m²,1989,Bestaande bouw,"Eengezinswoning, 2-onder-1-kapwoning",Zadeldak bedekt met pannen,5 kamers (3 slaapkamers),1 badkamer en 2 aparte toiletten,3 woonlagen,B,Aan rustige weg en in woonwijk,"Achtertuin, voortuin en zijtuin",2.190
4,Wildrijklaan 8,Sassenheim,€ 1.050.000,423 m²,210 m²,2000,Bestaande bouw,"Eengezinswoning, 2-onder-1-kapwoning",Schilddak bedekt met pannen,6 kamers (5 slaapkamers),2 badkamers en 2 aparte toiletten,4 woonlagen en een kelder,A,"Aan rustige weg, aan water, beschutte ligging,...","Achtertuin, voortuin en zijtuin",1.340


In [80]:
# simplify some of the columns
rename_map = {
    "Living space size (m2)": "living_area_m2",
    "Lot size (m2)": "lot_size_m2",
    "Estimated neighbourhood price per m2": "est_neigh_price_m2",
}
df = df.rename(columns=rename_map)


In [81]:
import re
import numpy as np

def parse_price(str):
    if pd.isna(str):
        return np.nan
    elif "prijs" in str:        # catches "Prijs op aanvraag" which in this case gets converted to a nan
        return np.nan
    elif '€' in str:
        return str.split('€')[1].replace('.', '')
    
    try:
        return float(str)
    except:
        return np.nan




In [94]:
df["Price"] = df["Price"].map(parse_price)

In [83]:
def parse_area(x):
    if pd.isna(x):
        return np.nan
    s = str(x).lower()
    s = s.replace("m²","")
    try:
        return float(s)
    except:
        return np.nan


In [84]:
df['living_area_m2']= df['living_area_m2'].map(parse_area)
df['lot_size_m2'] = df['lot_size_m2'].map(parse_area)

In [85]:
#separating bedrooms and total number of rooms from the 'Rooms' column
def parse_rooms(s):
    if pd.isna(s): return np.nan
    s = str(s).lower()
    match = re.search(r"(\d+)\s*kamer", s)
    return int(match.group(1)) if match else np.nan

def parse_bedrooms(s):
    if pd.isna(s): return np.nan
    s = str(s).lower()
    match = re.search(r"\((\d+)\s*slaap", s)
    return int(match.group(1)) if match else np.nan

df["rooms_total"] = df["Rooms"].map(parse_rooms)
df["bedrooms"]    = df["Rooms"].map(parse_bedrooms)

In [86]:
#taking the number of bathrooms

def parse_bathrooms(s):
    if pd.isna(s): return np.nan
    s = str(s).lower()
    match = re.search(r"(\d+)\s*badkamer", s)
    return int(match.group(1)) if match else np.nan

df["Bathrooms"] = df["Toilet"].map(parse_bathrooms)

In [87]:
#number of floors 

def parse_floors(s):
    if pd.isna(s): return np.nan
    match = re.search(r"(\d+)", str(s))
    return int(match.group(1)) if match else np.nan

df["Floors"] = df["Floors"].map(parse_floors)


In [88]:
import pandas as pd

STATLINE_PATH = "Woonplaatsen_in_Nederland_2024_21082025_161916.csv"

# load correctly: semicolon separator + quotes
lookup = pd.read_csv(
    STATLINE_PATH,
    sep=";",              # delimiter is semicolon
    skiprows=4            # skip the metadata/title rows at the top
)

lookup.rename(columns={"Woonplaatsen": "City", 'Naam' : 'Municipality', 'Naam.1' : 'Province'}, inplace=True)


In [89]:
city_to_province = dict(zip(lookup["City"], lookup["Province"]))

In [90]:
df["province_nl"] = df["City"].map(city_to_province)

In [ ]:
df["Build year"] = pd.to_numeric(df["Build year"], errors="coerce").astype("Int64")
df["price_per_sqm"] = df["Price"] / df['living_area_m2']


In [98]:
df.isna().sum()


Address                 0
City                    0
Price                  13
lot_size_m2             0
living_area_m2          0
Build year            104
Build type              0
House type              0
Roof                    0
Rooms                   0
Toilet                  0
Floors                  0
Energy label            0
Position              304
Garden                 58
est_neigh_price_m2    169
rooms_total             0
bedrooms                8
Bathrooms               0
province_nl           311
price_per_sqm          13
dtype: int64

In [99]:
df.to_csv("housing_clean.csv", index=False)
